In [26]:
import json, re
from collections import defaultdict

In [27]:
def to_utc(date):
    def trnsl(month):
        months = ["ledna", "února", "března", "dubna", "května", "června", "července", "srpna", "září", "října", "listopadu", "prosince"]
        return months.index(month) + 1
    date_parts = date.split()
    return date_parts[2] + "-" \
        + str(trnsl(date_parts[1])).zfill(2) + "-" \
        + date_parts[0].strip(".").zfill(2)

In [46]:
with open("data.json", encoding="utf8") as file:
    smich_data = json.load(file)

In [48]:
for smich in smich_data:
    smich_count = len(re.findall("\(.*?[sS]mích.*?\)", smich["speech"]))
    smich["haha"] = smich_count
    smich["wc"] = len(smich["speech"])
    smich["date"] = to_utc(smich["date"])
    del smich["speaker"]
    del smich["link"]
    del smich["speech"]

In [99]:
# normalize
for smich in smich_data:
    smich["hahaNorm"] = round(smich["haha"]/smich["wc"]*(10**4))

In [122]:
# merge
data = {date: defaultdict(int) for date in set([el["date"] for el in smich_data])}
for smich in smich_data:
    for el in ["haha", "wc", "hahaNorm"]:
        data[smich["date"]][el] += smich[el]

In [123]:
with open("results_timeline.json","w",encoding="utf8") as file:
    json.dump(data, file, ensure_ascii=False)

In [124]:
with open("results_timeline_hahaindex.json","w",encoding="utf8") as file:
    json.dump([[date, data[date]["hahaNorm"]] for date in data], file, ensure_ascii=False)

In [125]:
[[date, data[date]["hahaNorm"]] for date in data]

[['2012-06-05', 851],
 ['2015-09-15', 550],
 ['2018-03-07', 18],
 ['2016-05-24', 356],
 ['2013-12-04', 92],
 ['2014-09-02', 1],
 ['2019-05-29', 5],
 ['2017-05-16', 305],
 ['2018-07-11', 194],
 ['2013-08-16', 26],
 ['2011-11-04', 314],
 ['2018-12-04', 852],
 ['2019-07-09', 391],
 ['2014-02-04', 356],
 ['2010-12-07', 1194],
 ['2013-03-19', 289],
 ['2018-05-22', 1276],
 ['2012-03-13', 427],
 ['2011-02-01', 500],
 ['2015-03-10', 1168],
 ['2018-09-11', 420],
 ['2014-01-21', 158],
 ['2015-10-01', 19],
 ['2010-08-10', 1],
 ['2017-09-05', 214],
 ['2011-09-22', 3],
 ['2015-06-16', 618],
 ['2015-06-18', 579],
 ['2010-12-21', 294],
 ['2013-02-05', 278],
 ['2017-12-12', 366],
 ['2016-04-12', 593],
 ['2012-07-18', 72],
 ['2012-04-24', 673],
 ['2011-08-30', 221],
 ['2013-12-06', 16],
 ['2018-03-09', 80],
 ['2017-10-16', 12],
 ['2019-05-28', 611],
 ['2014-11-04', 30],
 ['2013-06-11', 231],
 ['2014-09-16', 478],
 ['2018-08-22', 41],
 ['2015-01-20', 1064],
 ['2014-04-29', 901],
 ['2016-06-28', 743],
 [